In [1]:
import multiprocessing
from langchain_community.chat_models import ChatLlamaCpp


# Path to your model weights
local_model = "../models/Hermes-2-Pro-Llama-3-8B-Q8.gguf"


llm = ChatLlamaCpp(
    temperature=0.5,
    model_path=local_model,
    n_ctx=10000,
    n_gpu_layers=8,
    n_batch=300,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    max_tokens=512,
    n_threads=multiprocessing.cpu_count() - 1,
    repeat_penalty=1.5,
    top_p=0.5,
    verbose=True,
)

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from ../models/Hermes-2-Pro-Llama-3-8B-Q8.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Hermes-2-Pro-Llama-3-8B
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.attent

In [2]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]

ai_msg = llm.invoke(messages)
ai_msg

llama_perf_context_print:        load time =    1432.49 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    34 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     5 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    2169.29 ms /    39 tokens


AIMessage(content="J'aime programmer.", additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run-d07d75af-92a7-44ac-86d8-992cc1f04161-0')